In [1]:
!pip install -q transformers==4.30.0 datasets scikit-learn pandas

import pandas as pd
from datasets import load_dataset
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score

print(">>> Loading Balanced Dataset (50 Toxic / 50 Non-toxic)...")

dataset_stream = load_dataset("google/civil_comments", split="train", streaming=True)

toxic_ds = list(dataset_stream.filter(lambda x: x['toxicity'] >= 0.5).take(50))
nontoxic_ds = list(dataset_stream.filter(lambda x: x['toxicity'] < 0.5).take(50))

dataset = toxic_ds + nontoxic_ds
data = pd.DataFrame(dataset)

data = data.sample(frac=1, random_state=42).reset_index(drop=True)

data['label'] = data['toxicity'].apply(lambda x: 1 if x >= 0.5 else 0)

print(f"Data Loaded: {len(data)} examples (Balanced: {data['label'].sum()} Toxic)")

def naive_baseline(text):
    text = text.lower()
    bad_words = ["stupid", "idiot", "dumb", "shut up", "trash", "kill", "ugly", "fat", "hell", "crazy"]
    if any(w in text for w in bad_words):
        return 1
    else:
        return 0

data['baseline_pred'] = data['text'].apply(naive_baseline)

print(">>> Running AI Pipeline (approx. 1-2 min)...")

toxicity_model = pipeline("text-classification", model="s-nlp/roberta_toxicity_classifier")

def run_ai(text):
    truncated_text = text[:512]
    result = toxicity_model(truncated_text)[0]

    if result['label'] == 'toxic':
        return 1
    elif result['label'] == 'neutral':
        return 0
    else:
        return 1 if result['score'] > 0.5 else 0

data['ai_pred'] = data['text'].apply(run_ai)

baseline_acc = accuracy_score(data['label'], data['baseline_pred'])
ai_acc = accuracy_score(data['label'], data['ai_pred'])

baseline_f1 = f1_score(data['label'], data['baseline_pred'])
ai_f1 = f1_score(data['label'], data['ai_pred'])

print("\n" + "="*40)
print(f" [Results] Baseline Accuracy:   {baseline_acc:.2f}")
print(f" [Results] AI Pipeline Accuracy: {ai_acc:.2f}")
print("-" * 40)
print(f" [Results] Baseline F1 Score:   {baseline_f1:.2f}")
print(f" [Results] AI Pipeline F1 Score: {ai_f1:.2f}")
print("="*40 + "\n")

print(">>> [Failure Cases: Baseline Failed vs. AI Succeeded]")
diff = data[(data['baseline_pred'] != data['label']) & (data['ai_pred'] == data['label'])]

for i, row in diff.head(3).iterrows():
    print(f"\n[Case Index {i}]")
    print(f"Comment: \"{row['text'][:200]}...\"")
    print(f" - True Label:      {'Toxic' if row['label']==1 else 'Non-toxic'}")
    print(f" - Baseline Pred:   {'Toxic' if row['baseline_pred']==1 else 'Non-toxic'} (Incorrect)")
    print(f" - AI Pipeline Pred:{'Toxic' if row['ai_pred']==1 else 'Non-toxic'} (Correct)")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 30.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
>>> Loading Balanced Dataset (50 Toxic / 50 Non-toxic)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Data Loaded: 100 examples (Balanced: 50 Toxic)
>>> Running AI Pipeline (approx. 1-2 min)...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



 [Results] Baseline Accuracy:   0.61
 [Results] AI Pipeline Accuracy: 0.86
----------------------------------------
 [Results] Baseline F1 Score:   0.40
 [Results] AI Pipeline F1 Score: 0.84

>>> [Failure Cases: Baseline Failed vs. AI Succeeded]

[Case Index 3]
Comment: "They are terrorists pure and simple...."
 - True Label:      Toxic
 - Baseline Pred:   Non-toxic (Incorrect)
 - AI Pipeline Pred:Toxic (Correct)

[Case Index 5]
Comment: "I honestly cannot decide if these guys are complete morons or the most patriotic hero's this country has seen in a long time...."
 - True Label:      Toxic
 - Baseline Pred:   Non-toxic (Incorrect)
 - AI Pipeline Pred:Toxic (Correct)

[Case Index 8]
Comment: "Their ridiculous band photo has me wanting to drink white wine at 3 in the afternoon.
It's almost 5, right?..."
 - True Label:      Toxic
 - Baseline Pred:   Non-toxic (Incorrect)
 - AI Pipeline Pred:Toxic (Correct)
